In [1]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import exc, create_engine
from bs4 import BeautifulSoup
from urllib.request import urlopen
import logging

In [2]:
def sql_connection():
    try:
        connection = create_engine('postgresql+psycopg2://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + '5432' + '/' + os.environ.get('RDS_DB'),
                                    connect_args = {'options': '-csearch_path=nba_source'},
                     echo = False)
        logging.info('SQL Connection Successful')
        print('SQL Connection Successful')
        return(connection)
    except exc.SQLAlchemyError as e:
        logging.info('SQL Connection Failed, Error:', e)
        print('SQL Connection Failed, Error:', e)
        return(e)

In [3]:
conn = sql_connection()

SQL Connection Successful


In [12]:
tables = pd.read_sql_query('SELECT table_name FROM information_schema.tables', conn)


In [2]:
conn = psycopg2.connect(host=os.environ.get('IP'), 
                 port="5432", 
                 user=os.environ.get('RDS_USER'), 
                 password=os.environ.get('RDS_PW'), 
                 database=os.environ.get('RDS_DB'), 
                 options="-c search_path=nba_prod") # setting schema

In [6]:
df = pd.read_sql_query('SELECT table_name FROM information_schema.tables', conn)

In [8]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df2 = pd.DataFrame(data=d)
df2

,col1,col2
0,1,3
1,2,4


In [6]:
def write_to_sql(data, table_type):
    data_name = [ k for k,v in globals().items() if v is data][0]
    if len(data) == 0:
        print(data_name + " Failed, not writing to SQL")
    else:
        # ^ this disgusting monstrosity is to get the name of the -fucking- dataframe lmfao
        data.to_sql(con = conn, name = ("aws_" + data_name + "_table"), index = False, if_exists = table_type)
        print("Writing aws_" + data_name + "_table to SQL")

In [13]:
df.to_sql(con = conn, name = 'aws_stats_table_prac', index = False, if_exists = 'replace')

In [4]:
def schedule_scraper(month):
    try:
        global schedule_df
        url = "https://www.basketball-reference.com/leagues/NBA_2022_games-{}.html".format(month)
        html = urlopen(url)
        soup = BeautifulSoup(html)

        headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]
        headers[6] = 'boxScoreLink'
        headers[7] = 'isOT'
        headers = headers[1:]

        rows = soup.findAll('tr')[1:]
        date_info = [[th.getText() for th in rows[i].findAll('th')]
                for i in range(len(rows))]

        game_info = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
        date_info = [i[0] for i in date_info]

        schedule = pd.DataFrame(game_info, columns = headers)
        schedule['Date'] = date_info

        schedule_df = schedule_df.append(schedule)
        logging.info(f'Schedule Function Completed for {month}, retrieving {len(schedule_df)} rows')
        print(f'Schedule Function Completed for {month}, retrieving {len(schedule_df)} rows')
    except ValueError:
        logging.info("Schedule Scraper Function Failed")
        print("Schedule Scraper Function Failed")
        df = []
        return(df)

month_list = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
schedule_df = pd.DataFrame()
for month in month_list:
    schedule_scraper(month)

Schedule Function Completed for october, retrieving 93 rows
Schedule Function Completed for november, retrieving 318 rows
Schedule Function Completed for december, retrieving 538 rows
Schedule Function Completed for january, retrieving 765 rows
Schedule Function Completed for february, retrieving 925 rows
Schedule Function Completed for march, retrieving 1150 rows
Schedule Function Completed for april, retrieving 1230 rows


In [5]:
from datetime import datetime, date

In [5]:
schedule_df = schedule_df[['Start (ET)', 'Visitor/Neutral', 'Home/Neutral', 'Date']]
schedule_df['proper_date'] = pd.to_datetime(schedule_df['Date']).dt.date
schedule_df.columns = schedule_df.columns.str.lower()
# schedule_df = schedule_df.rename(columns = {"Today": "team", "SPREAD": "spread", "TOTAL": "total_pts", "MONEYLINE": "moneyline"})

In [7]:
schedule_df = schedule_df.rename(columns = {"start (et)": "start_time", "visitor/neutral": "away_team", "home/neutral": "home_team"})

In [9]:
schedule_df.to_sql(con = conn, name = "aws_schedule_table", if_exists = 'replace')

In [10]:
def get_advanced_stats():
    try:
        url = "https://www.basketball-reference.com/leagues/NBA_2021.html"
        df = pd.read_html(url)
        df = pd.DataFrame(df[10])
        df.drop(columns = df.columns[0], 
            axis=1, 
            inplace=True)

        df.columns = ['Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORTG', 'DRTG', 'NRTG', 'Pace', 'FTr', '3PAr', 'TS%', 'bby1', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'bby2', 'eFG%_opp', 'TOV%_opp', 'DRB%_opp', 'FT/FGA_opp', 'bby3', 'Arena', 'Attendance', 'Att/Game']
        df.drop(['bby1', 'bby2', 'bby3'], axis = 1, inplace = True)
        df = df.query('Team != "League Average"')
        df['Team'] = df['Team'].str.replace("*", "")
        df.columns = df.columns.str.lower()
        logging.info(f'Advanced Stats Function Successful, retrieving updated data for 30 Teams')
        print(f'Advanced Stats Function Successful, retrieving updated data for 30 Teams')
        return(df)
    except ValueError:
        logging.info("Advanced Stats Function Failed for Today's Games")
        print("Advanced Stats Function Failed for Today's Games")
        df = []
        return(df)

In [11]:
df = get_advanced_stats()

Advanced Stats Function Successful, retrieving updated data for 30 Teams


<ipython-input-10-a655e1138752>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Team'] = df['Team'].str.replace("*", "")
<ipython-input-10-a655e1138752>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Team'] = df['Team'].str.replace("*", "")
